It runs and moves in order from the top.
Settings include model changes.
>上から順に実行して動きます。
>設定はモデル変更などがあります。

Adapted from（参考元）: 

https://github.com/camenduru/stable-diffusion-webui-colab/

In [ ]:
#@title Clone webUI repository, etc.（webUIリポジトリのクローンなど）
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui --quiet
%cd /content/stable-diffusion-webui

!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl

In [ ]:
#@title Download the model from Hugging Face.（Hugging Faceからモデルをダウンロード）
%cd /content/stable-diffusion-webui

import os
import io
import json
import requests
import pandas as pd

def resolve_raw_url(url):
  if url.startswith("https://huggingface.co/"):
    url = url.replace("/blob/", "/resolve/")
  elif url.startswith("https://github.com/"):
    url = url.replace("/blob/", "/raw/")
  return url

url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/configs/model-config.csv")
response = requests.get(url, timeout=10)
models_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), header=0, index_col=0)

vaes_dict = {
  "None" : "",
  "trinart-derrida_final" : "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/blob/main/autoencoder_fix_kl-f8-trinart_characters.ckpt",
  "waifu-v14-anime2" : "https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/vae/kl-f8-anime2.ckpt",
  "sd-vae-ft-mse" : "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.ckpt",
}

localizationes_dict = {
  "None" : "",
  "ar_AR" : "https://github.com/AUTOMATIC1111/stable-diffusion-webui-old-localizations",
  "de_DE" : "https://github.com/Strothis/stable-diffusion-webui-de_DE",
  "es_ES" : "https://github.com/innovaciones/stable-diffusion-webui-localization-es_ES",
  "fr-FR" : "https://github.com/AUTOMATIC1111/stable-diffusion-webui-old-localizations",
  "it_IT" : "https://github.com/Harvester62/stable-diffusion-webui-localization-it_IT",
  "ja_JP" : "https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP",
  "ko_KR" : "https://github.com/36DB/stable-diffusion-webui-localization-ko_KR",
  "no_NO" : "https://github.com/Cyanz83/stable-diffusion-webui-localization-no_NO",
  "pt_BR" : "https://github.com/M-art-ucci/stable-diffusion-webui-localization-pt_BR",
  "ru_RU" : "https://github.com/ProfaneServitor/stable-diffusion-webui-localization-ru_RU",
  "tr_TR" : "https://github.com/camenduru/stable-diffusion-webui-localization-tr_TR",
  "zh_CN" : "https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN",
  "zh_TW" : "https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW",
}

model = "waifu-v14-e1" #@param ['None', 'waifu-v14-e1', 'stable-v21-768', 'stable-v20-512-inpainting', 'stable-v21-512', 'stable-v20-512-depth', 'cool-japan', 'mitsua-v10', 'waifu-v140-booru-14000', 'robo-v2', 'ACertainThing', 'ACertainModel', 'ACertainty', 'stable-v15', 'stable-v15-inpainting', 'plat-v122', '8528-final', 'waifu-v135-80000', 'waifu-v130-float16', 'sd-wikiart-v2', 'artstation-v1', 'cafe-instagram-unofficial', 'trinart-derrida_final', 'robo-v1']
vae = "None" #@param ['None', 'trinart-derrida_final', 'waifu-v14-anime2', 'sd-vae-ft-mse']
localization = "ja_JP" #@param ['None', 'ar_AR', 'de_DE', 'es_ES', 'fr-FR', 'it_IT', 'ja_JP', 'ko_KR', 'no_NO', 'pt_BR', 'ru_RU', 'tr_TR', 'zh_CN', 'zh_TW']

if model != 'None':
  model_url = resolve_raw_url(models_df.at[model, 'url'])
  !wget {model_url} -P /content/stable-diffusion-webui/models/Stable-diffusion/
  model_basename = os.path.splitext(os.path.basename(model_url))[0]
  config_url = resolve_raw_url(models_df.at[model, 'config_url'])
  !wget -q {config_url} -O /content/stable-diffusion-webui/models/Stable-diffusion/{model_basename}.yaml

if vae != 'None':
  vae_url = resolve_raw_url(vaes_dict[vae])
  !wget {vae_url} -P /content/stable-diffusion-webui/models/VAE/

if localization != 'None':
  localization_url = localizationes_dict[localization]
  %cd /content/stable-diffusion-webui/extensions
  !git clone {localization_url} --quiet
  %cd /content/stable-diffusion-webui

styles_url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/styles.csv")
!wget -q {styles_url} -O /content/stable-diffusion-webui/styles.csv

config_url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/configs/config.json")
response = requests.get(config_url, timeout=10)
config_json = response.json()
config_json["localization"] = localization
config_json["CLIP_stop_at_last_layers"] = int(models_df.at[model, 'clip_stop_at_last_layers'])
with open('/content/stable-diffusion-webui/config.json', "w", encoding="utf8") as file:
  json.dump(config_json, file, indent=4)

ui_config_url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/configs/ui-config.json")
response = requests.get(ui_config_url, timeout=10)
ui_config_json = response.json()
ui_config_json["txt2img/Width/value"]  = int(models_df.at[model, 'width_height_value'])
ui_config_json["txt2img/Height/value"] = int(models_df.at[model, 'width_height_value'])
ui_config_json["img2img/Width/value"]  = int(models_df.at[model, 'width_height_value'])
ui_config_json["img2img/Height/value"] = int(models_df.at[model, 'width_height_value'])
ui_config_json["extras/Width/value"]   = int(models_df.at[model, 'width_height_value'])
ui_config_json["extras/Height/value"]  = int(models_df.at[model, 'width_height_value'])
with open('/content/stable-diffusion-webui/ui-config.json', "w", encoding="utf8") as file:
  json.dump(ui_config_json, file, indent=4)


In [ ]:
#@title Download the model from Google Drive.（Google Driveからモデルをダウンロード）
%cd /content/stable-diffusion-webui

import os
import glob
from google.colab import drive # type: ignore

use_google_drive = False #@param {type:"boolean"}
sd_webui_gdrive = "/content/drive/MyDrive/sd/stable-diffusion-webui" #@param {type:"string"}
sd_webui_colab = "/content/stable-diffusion-webui"

create_directory_gdrive = False #@param {type:"boolean"}
#@markdown Find and copy the following files from the stable-diffusion-webui(sd_webui_gdrive) directory.
#@markdown > stable-diffusion-webui(sd_webui_gdrive)ディレクトリから以下のファイルを見つけてコピーします。

#@markdown + models/Stable-diffusion/\*.ckpt
#@markdown + models/Stable-diffusion/\*.yaml
#@markdown + models/VAE/\*.ckpt
#@markdown + models/VAE/\*.pt
#@markdown + models/Stable-diffusion/\*.vae.ckpt
#@markdown + models/Stable-diffusion/\*.vae.pt
#@markdown + embeddings/\*.pt
#@markdown + embeddings/\*.bin
#@markdown + models/hypernetworks/*.pt
#@markdown + extensions/aesthetic-gradients/aesthetic_embeddings/*.pt
#@markdown + styles.csv
#@markdown + config.json
#@markdown + ui-config.json
#@markdown + /content/drive/MyDrive/model.ckpt
#@markdown + /content/drive/MyDrive/AI/models/*.ckpt

model_checkpoints = "models/Stable-diffusion"
model_model_vaes = "models/Stable-diffusion"
model_vae_vaes = "models/VAE"
textual_inversion_embeddings = "embeddings"
hypernetworks = "models/hypernetworks"
aesthetic_embeddings = "extensions/aesthetic-gradients/aesthetic_embeddings"
config_list = ["styles.csv", "config.json", "ui-config.json"]

model_checkpoints_gdrive = os.path.join(sd_webui_gdrive, model_checkpoints)
model_model_vaes_gdrive = os.path.join(sd_webui_gdrive, model_model_vaes)
model_vae_vaes_gdrive = os.path.join(sd_webui_gdrive, model_vae_vaes)
textual_inversion_embeddings_gdrive = os.path.join(sd_webui_gdrive, textual_inversion_embeddings)
hypernetworks_gdrive = os.path.join(sd_webui_gdrive, hypernetworks)
aesthetic_embeddings_gdrive = os.path.join(sd_webui_gdrive, aesthetic_embeddings)

model_checkpoints_colab = os.path.join(sd_webui_colab, model_checkpoints)
model_model_vaes_colab = os.path.join(sd_webui_colab, model_model_vaes)
model_vae_vaes_colab = os.path.join(sd_webui_colab, model_vae_vaes)
textual_inversion_embeddings_colab = os.path.join(sd_webui_colab, textual_inversion_embeddings)
hypernetworks_colab = os.path.join(sd_webui_colab, hypernetworks)
aesthetic_embeddings_colab = os.path.join(sd_webui_colab, aesthetic_embeddings)

if use_google_drive:
  drive.mount("/content/drive")

  ckpt_path = "/content/drive/MyDrive/model.ckpt"
  if os.path.exists(ckpt_path):
    !cp {ckpt_path} {model_checkpoints_colab}
  for model_checkpoint in glob.glob(f"/content/drive/MyDrive/AI/models/*.ckpt"):
    !cp {model_checkpoint} {model_checkpoints_colab}
 
  for config in config_list:
    config_gdrive = os.path.join(sd_webui_gdrive, config)
    if os.path.exists(config_gdrive):
      !cp {config_gdrive} {sd_webui_colab}

  if create_directory_gdrive:
    os.makedirs(model_checkpoints_gdrive, exist_ok=True)
    os.makedirs(model_model_vaes_gdrive, exist_ok=True)
    os.makedirs(model_vae_vaes_gdrive, exist_ok=True)
    os.makedirs(textual_inversion_embeddings_gdrive, exist_ok=True)
    os.makedirs(hypernetworks_gdrive, exist_ok=True)
    os.makedirs(aesthetic_embeddings_gdrive, exist_ok=True)
  os.makedirs(hypernetworks_colab, exist_ok=True)

  files = glob.glob(f"{model_checkpoints_gdrive}/*.ckpt")
  files.extend(glob.glob(f"{model_checkpoints_gdrive}/*.yaml"))
  for model_checkpoint in files:
    !cp {model_checkpoint} {model_checkpoints_colab}

  files = glob.glob(f"{model_model_vaes_gdrive}/*.vae.ckpt")
  files.extend(glob.glob(f"{model_model_vaes_gdrive}/*.vae.pt"))
  for model_vae in files:
    !cp {model_vae} {model_model_vaes_colab}
    
  files = glob.glob(f"{model_vae_vaes_gdrive}/*.ckpt")
  files.extend(glob.glob(f"{model_vae_vaes_gdrive}/*.pt"))
  for model_vae in files:
    !cp {model_vae} {model_vae_vaes_colab}

  files = glob.glob(f"{textual_inversion_embeddings_gdrive}/*.pt")
  files.extend(glob.glob(f"{textual_inversion_embeddings_gdrive}/*.bin"))
  for textual_inversion_embedding in files:
    !cp {textual_inversion_embedding} {textual_inversion_embeddings_colab}

  for hypernetwork in glob.glob(f"{hypernetworks_gdrive}/*.pt"):
    !cp {hypernetwork} {hypernetworks_colab}
    
  for aesthetic_embedding in glob.glob(f"{aesthetic_embeddings_gdrive}/*.pt"):
    !cp {aesthetic_embedding} {aesthetic_embeddings_colab}

In [ ]:
#@title Launch the webUI.（webUIの起動）
%cd /content/stable-diffusion-webui

# RAM不足のため
import gc
from google.colab import drive
drive.flush_and_unmount()
gc.collect()

username = "user" #@param {type:"string"}
password = "pass" #@param {type:"string"}
tunnel = "gradio" #@param ["ngrok", "gradio"]
#@markdown get a token from for ngrok. https://dashboard.ngrok.com/get-started/your-authtoken （ngrokからトークンを取得）
ngrok_authtoken = "" #@param {type:"string"}
ngrok_region = "jp" #@param ["us", "eu", "au", "ap", "sa", "jp", "in"]


common_args = f"--xformers --lowram --enable-console-prompts --enable-insecure-extension-access --disable-safe-unpickle --update-check --gradio-auth {username}:{password}"
if tunnel == "ngrok":
  !ATTN_PRECISION=fp16 COMMANDLINE_ARGS="{common_args} --ngrok {ngrok_authtoken} --ngrok-region {ngrok_region}" python launch.py
else:
  !ATTN_PRECISION=fp16 COMMANDLINE_ARGS="{common_args} --share" python launch.py


In [ ]:
#@title Save to zip file with same password as cell above. （上のセルと同じパスワードでzipファイルに保存）

%cd /content/stable-diffusion-webui
!zip -er --password={password} outputs.zip outputs 

In [ ]:
#@title Save the zip file to Google Drive. （Google Driveにzipファイルを保存）
from google.colab import drive 
drive_path = "/content/drive"
drive.mount(drive_path, force_remount=False)

!cp "/content/stable-diffusion-webui/outputs.zip" "/content/drive/MyDrive"